# Etude d'un système de spheres dures par simulation moléculaire

In [ ]:
import numpy,pandas
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import os

## Equation d'état à basse densité -- Deuxième coefficient du viriel

Nous allons ici comparer la pression donnée par la simulation avec celle connue analytiquement. Nous réalisons une simulation d'un gaz de sphères dures de température $T=1$, de densité $n=0.00022$ et constitué de $N=108$ particules, placées initialement dans une boîte cubique comprenant 3x3x3 mailles fcc.

Nous travaillons dans le système d'unités où le rayon des sphères vaut $1$, la masse des particules vaut $1$ et la constante de Boltzmann $k=1$.

La pression doit s'exprimer en fonction de la densité comme suit

$$ \frac {P}{nT} = 1 + B_2 n + B_3 n^2 + \mathcal{O}(n^3) $$

Les deuxième et troisième coefficients du viriel sont donnés par

$$ B_2 = \frac{16\pi R^3}{3} = \frac {16\pi}{3} = 16.755 $$
$$ B_3 = \frac{160\pi^2 R^6}{9} = \frac{160\pi}{9} = 175.460 $$

Les termes correspondant valent

$$ B_2 n = 16.755 \times 0.00022 = 3.686 \times 10^{-3} $$
$$ B_3 n^2 = 175.460 \times 0.00022^2 = 8.492 \times 10^{-6} $$

Nous négligerons la contribution des coefficients d'ordre 3 et plus. 

L'équation d'état que nous devrions trouver numériquement pour la pression est donc 

$$ \frac {P}{nT} = 1.00369(1) $$

### Equation d'état à partir d'une longue simulation

In [ ]:
os.system('./simulationO3 3 3 3 0.00022 1 21000 0')
os.system("mv data/collisionData.csv data/collisionData_long.csv".format(i))
os.system("mv data/infoSimulation.csv data/infoSimulation_long.csv".format(i))

In [ ]:
#collisionData = pandas.read_csv('data/collisionData_3_3_3_0.00022_1_100000_0_0.02_5_6.dat')
collisionData = pandas.read_csv('data/collisionData_long.csv')
[t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

print (t[-1])

In [ ]:
N = 108
T = 1
Delta_t = 1000

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

In [ ]:
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("p avec petit Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

In [ ]:
p_moy = numpy.sum(p)/len(p)
print (p_moy)

#### Estimation de l'erreur en considérant plusieurs les subdivisions $\Delta t$ de ce temps comme indépendantes

Attention: nous ne savons pas si cette hypothèse est valide, nous comparerons plus loin cette erreur à celle obtenue en considérant plusieurs simulations différentes.

In [ ]:
p_err = numpy.std(p)/numpy.sqrt(len(p))
print (p_err)

### Calcul du temps de corrélation

La détermination du temps de corrélation est importante pour connaître le durée $\Delta t$ à utiliser pour calculer la moyenne et l'erreur sur la pression du système. En effet, les valeurs de la pression calculées sur un intervalle $\Delta t$ trop petit sont corrélées entre elles ce qui complique le calcul de l'erreur statistique sur la pression moyenne. Le temps de corrélation $\tau$ d'une observable $A$ est défini à partir de la fonction de corrélation  $c_{AA}$:

$$ \tau = \frac 12 \sum_{n=-\infty}^{+\infty} \frac{c_{AA}(n)}{c_{AA}(0)} $$
$$ c_{AA}(k) = \langle A_n A_{n+k} \rangle - \langle A_n \rangle ^2 $$

L'observable en question est la "pression réduite" que l'on a défini comme:

$$ p_n = \frac{\beta P_n}{n} = 1 + \frac{1}{3 N T \Delta t} \sum v_{ij}\cdot r_{ij} \qquad \text{somme sur les collisions durant [}n\Delta t\text{ , } (n+1)\Delta t \text{[} $$

In [ ]:
def c_pp(k,p):
    
    N = len(p)
    
    if k>=0:
        avg1 = numpy.sum(p[0:N-k]*p[k:N]) / (N-k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[0:N-k]) / (N-k) # < p_n >
    elif k<0:
        avg1 = numpy.sum(p[-k:N]*p[0:N+k]) / (N+k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[-k:N]) / (N+k) # < p_n >
    
    return avg1-avg2**2

In [ ]:
N = 108
T = 1
Delta_t = 1000

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

On vérifie que $c_{pp}(0)$ est bien la variance du vecteur des pressions réduites.

In [ ]:
print( c_pp(0,p) )
print( numpy.std(p)**2 )

<span style="color:red"> La fonction de corrélation semble beaucoup trop grande. Il me semble qu'elle est censée décroître quand $k$ s'éloigne de $0$. Est-ce donc une erreur ou alors le système est corrélé sur des temps très longs ? </span>

In [ ]:
for i in range(int(len(p)/2)):
    print( "c_pp({:d},p) = {:.3f}e-06".format(i,c_pp(i,p)*1e6) )

### Execution de plusieurs simulations pour avoir des données indépendantes

On exécute les commandes linux servant à effectuer les simulations avec le module "os".

In [ ]:
NSim = 20 # nombre de simulations

for i in range(NSim):
    os.system('./simulationO3 3 3 3 0.00022 1 1000 0')
    os.system("mv data/collisionData.csv data/collisionData{:d}.csv".format(i))
    os.system("mv data/infoSimulation.csv data/infoSimulation{:d}.csv".format(i))

In [ ]:
N = 108
T = 1
Delta_t = 100

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data/collisionData{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
    for j in range(len(t)):
        p[int((t[j]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[j]

    p = p[:-1] # dernier bin incomplet

    p_sim[i] = numpy.sum(p)/len(p)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("p pour plusieurs simulations");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)